# Tasks

> FrankenUI Tasks Example

In [1]:
#| default_exp tasks

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from fasthtml.common import *
from fh_frankenui.components import *
from fasthtml.svg import *
import json

## Data

We store data in Python to minic the static data in the examples.  Eventually this should be replaced with sqlite data store to allow for interactive functionality

In [4]:
#| export
with open('../data/status_list.json', 'r') as f: data     = json.load(f)
with open('../data/statuses.json',    'r') as f: statuses = json.load(f)

In [5]:
#| export
priority_dd = [{'priority': "low", 'count': 36 }, {'priority': "medium", 'count': 33 }, {'priority': "high", 'count': 31 }]

rows_per_page_dd = [10,20,30,40,50]

status_dd = [{'status': "backlog", 'count': 21 },{'status': "todo", 'count': 21 },{'status': "progress", 'count': 20 },{'status': "done",'count': 19 },{'status': "cancelled", 'count': 19 }]


## Tasks Page Functions

In [6]:
#| export
hotkeys = (('Profile','⇧⌘P'),('Billing','⌘B'),('Settings','⌘S'),('New Team',''),('Logout',''))
avatar_opts =Ul(cls='uk-dropdown-nav uk-nav')(
                            Li(cls='px-2 py-1.5 text-sm')(
                                Div(cls='flex flex-col space-y-1')(
                                    P('sveltecult', cls='text-sm font-medium leading-none'),
                                    P('leader@sveltecult.com', cls='text-xs leading-none text-muted-foreground'))),
                            Li(cls='uk-nav-divider'),
                            *map(lambda x: Li(A(cls='uk-drop-close justify-between')(x[0],Span(x[1],cls='ml-auto text-xs tracking-widest opacity-60'))), hotkeys))

In [52]:
#| export
def UkIconButton(*c, sz='small', cls=()):
    if sz not in ('small','medium','large'): raise ValueError(f"Invalid size '{sz}'. Must be 'small', 'medium', or 'large'.")
    return Button(cls=f'uk-icon-button uk-icon-button-{sz} ' + stringify(cls))(*c)

In [18]:
#| export
def CreateTaskModal():
    return Modal(
        Div(cls='p-6')(
            UkModalTitle('Create Task'),P(cls=TextT.muted_sm)('Fill out the information below to create a new task'),Br(),
            Form(cls='space-y-6')(
                Div(cls='grid grid-cols-3 gap-2')(
                    UkSelect(*Options('Documentation', 'Bug', 'Feature'), label='Task Type', id='task_type'),
                    UkSelect(*Options('In Progress', 'Backlog', 'Todo', 'Cancelled', 'Done'), label='Status', id='task_status'),
                    UkSelect(*Options('Low', 'Medium', 'High'), label='Priority', id='task_priority')),
                UkTextArea(label='Title', placeholder='Please describe the task that needs to be completed'),
                Div(cls='flex justify-end space-x-2')(
                    UkButton(cls=UkButtonT.ghost + ' uk-modal-close')('Cancel'),
                    UkButton(cls=UkButtonT.primary + ' uk-modal-close')('Submit')))),
        id='TaskForm')

In [19]:
#| export
page_heading =Div(cls='flex items-center justify-between space-y-2')(
            Div(cls='space-y-2')(
                UkH2('Welcome back!'),P("Here's a list of your tasks for this month!", cls=TextT.muted_sm)),
            Div(A(href='#', cls='h-8 w-8 inline-flex rounded-full bg-accent ring-ring')(Img(src='https://api.dicebear.com/8.x/lorelei/svg?seed=sveltecult')),
                Div(uk_dropdown='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop')(avatar_opts)))

In [29]:
#| export
table_controls =(UkInput(cls='w-[250px]',placeholder='Filter task'),
                 UkDropdownButton(label = "Status", 
                     options = list(A(FullySpacedContainer(a['status'], a['count'], wrap_tag=P),cls='capitalize') for a in status_dd),
                     btn_cls=(TextT.medium_xs, 'uk-button-default')),
                 UkDropdownButton(label = "Priority", 
                    options = [A(FullySpacedContainer(a['priority'], a['count'],wrap_tag=P),cls='capitalize') for a in priority_dd],
                     btn_cls=(TextT.medium_xs,'uk-button-default')),
                UkDropdownButton(label='View', 
                                 options = [FullySpacedContainer(P(o), UkIcon('check')) for o in ['Title','Status','Priority']]),
                UkButton('Create Task',cls=('uk-button-primary', TextT.medium_xs), uk_toggle="target: #TaskForm"))

## Route Data

In [ ]:
#| export
def tasks():
    return CreateTaskModal(),Div(cls='p-8')(
            page_heading,
        Div(cls='mt-8 flex items-center justify-between')(
                 Div(cls='flex flex-1 gap-4')(table_controls)),
             Div(cls='uk-overflow-auto mt-4 rounded-md border border-border')(
                 Table(cls='uk-table uk-table-middle uk-table-divider uk-table-hover uk-table-small')(
                     # Table header
                     Thead(Tr(Th(cls='uk-table-shrink p-2')(Input(type='checkbox', cls='uk-checkbox')),
                             *[Th(cls='p-2')(o) for o in ['Task','Title','Status','Priority']],
                             Th(cls='uk-table-shrink p-2'))),
                     # Table Body (Tasks content)
                     Tbody(
                         *[Tr(cls=a['selected'])(
                             Td(cls='uk-table-shrink p-2')(CheckboxX(cls='uk-checkbox')), 
                             Td(a['id'], cls='p-2'), # task id
                             Td(cls='uk-table-expand max-w-[500px] truncate p-2')(Span(a['label'], cls='uk-label capitalize'),Span(a['title'], cls='font-medium')),
                             *map(lambda x: Td(cls='uk-text-nowrap p-2 uk-text-capitalize')(x), (a['status'],a['priority'])) ,
                             Td(cls='p-2')(
                                 UkIconButton(cls='uk-icon-button-xsmall')(Span(uk_icon='more', cls='text-gray-400')),
                                 Div(uk_dropdown='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop')(
                                     Ul(cls='uk-dropdown-nav uk-nav')(
                                         *map(lambda x: Li(A(x, cls='uk-drop-close justify-between')), ('Edit','Make a copy','Favorite', ('Delete',Span('⌘⌫', cls='ml-auto text-xs tracking-widest opacity-60'))))))))
                           for a in data])),
             # Table Footer
             Div(cls='mt-4 flex items-center justify-between px-2')(
                 Div('1 of 100 row(s) selected.', cls='flex-1 text-sm text-muted-foreground'),
                 Div(cls='flex flex-none items-center space-x-8')(
                     Div('Page 1 of 10', cls='flex w-[100px] items-center justify-center text-sm font-medium'),
                     Div(cls='flex items-center space-x-2')(
                         # chevron navigation (dbl left, left, right, dbl right)
                         UkIconButton(cls='hidden lg:inline-flex')(Span('Go to last page', cls='sr-only'),
                             Span(uk_icon='chevron-double-left', cls='h-4 w-4')),
                         UkIconButton(Span('Go to previous page', cls='sr-only'),
                             Span(uk_icon='chevron-left', cls='h-4 w-4')),
                         UkIconButton(Span('Go to next page', cls='sr-only'),
                             Span(uk_icon='chevron-right', cls='h-4 w-4')),
                         UkIconButton(cls='hidden lg:inline-flex')(
                             Span('Go to last page', cls='sr-only'),
                             Span(uk_icon='chevron-double-right', cls='h-4 w-4')))))))

In [ ]:
#| export
tasks_homepage = tasks()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()